# 1. Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from scipy import stats

# Visualisations
#import missingno
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Regression models
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import AdaBoostRegressor

# Model slection
from sklearn.model_selection import train_test_split

# Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


# Other
from scipy import stats
import math
import pickle

# 2. EDA

In [2]:
train_df = pd.read_csv('df-train_set.csv')
test_df = pd.read_csv('df-test_set.csv')

test_ID = test_df['Index']
test_df = test_df.drop('Index', axis=1)

In [3]:
train_df = train_df[(train_df['Commodities'] == 'APPLE GOLDEN DELICIOUS')]

In [4]:
train_df.head()

,Province,Container,Size_Grade,Weight_Kg,Commodities,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
1,CAPE,M4183,1L,18.3,APPLE GOLDEN DELICIOUS,2020-09-09,150.0,170.0,51710.0,332,6075.6,822,8.51
7,CAPE,JG110,2M,11.0,APPLE GOLDEN DELICIOUS,2020-04-14,50.0,50.0,16000.0,320,3520.0,0,4.55
24,W.CAPE-BERGRIVER ETC,JE090,2S,9.0,APPLE GOLDEN DELICIOUS,2020-04-16,55.0,55.0,990.0,18,162.0,1506,6.11
40,CAPE,M4183,1S,18.3,APPLE GOLDEN DELICIOUS,2020-05-04,80.0,120.0,32020.0,388,7100.4,443,4.51
69,EASTERN CAPE,IA400,1S,400.0,APPLE GOLDEN DELICIOUS,2020-09-28,1800.0,1800.0,1800.0,1,400.0,2,4.50


In [5]:
test_df.head()

,Province,Container,Size_Grade,Weight_Kg,Commodities,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand
0,W.CAPE-BERGRIVER ETC,EC120,1M,12.0,APPLE GOLDEN DELICIOUS,2020-07-09,128.0,136.0,5008.0,38,456.0,0
1,W.CAPE-BERGRIVER ETC,M4183,1X,18.3,APPLE GOLDEN DELICIOUS,2020-01-20,220.0,220.0,1760.0,8,146.4,2
2,W.CAPE-BERGRIVER ETC,EC120,1S,12.0,APPLE GOLDEN DELICIOUS,2020-08-19,120.0,120.0,720.0,6,72.0,45
3,W.CAPE-BERGRIVER ETC,M4183,1M,18.3,APPLE GOLDEN DELICIOUS,2020-05-06,160.0,160.0,160.0,1,18.3,8
4,W.CAPE-BERGRIVER ETC,M4183,1L,18.3,APPLE GOLDEN DELICIOUS,2020-05-04,140.0,160.0,14140.0,100,1830.0,19


In [ ]:
#Changing date for train
train_df['Date']= pd.to_datetime(train_df['Date'])
#train_df['Date'] = train_df['Date'].dt.strftime('%d/%m/%Y')
train_df['Months'] = train_df['Date'].dt.strftime('%B')
train_df['Year'] = train_df['Date'].dt.strftime('%Y')


#Changing date for train
test_df['Date']= pd.to_datetime(test_df['Date'])
#train_df['Date'] = train_df['Date'].dt.strftime('%d/%m/%Y')
test_df['Months'] = test_df['Date'].dt.strftime('%B')
test_df['Year'] = test_df['Date'].dt.strftime('%Y')

In [ ]:
#Date
train_df['year'] = pd.DatetimeIndex(train_df['Date']).year
train_df['Months'] = pd.DatetimeIndex(train_df['Date']).month

In [ ]:
#Changing date for train
train_df['Date']= pd.to_datetime(train_df['Date'])

train_df['Date'] = train_df['Date'].dt.strftime('%B')

In [ ]:
#Changing date for test
test_df['Date']= pd.to_datetime(test_df['Date'])
test_df['Date']= test_df['Date'].dt.strftime('%B')

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
#np.percentile(train_df['Weight_Kg'], [25, 50, 75])

In [ ]:
train_df

In [ ]:
train_df.dtypes

In [ ]:
test_df.dtypes

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.shape

# 2 Future Selection 

1. we will look at the distribution of our target variable which is the Avarage Price Per KG

In [ ]:
# target variable distribution
sns.distplot(train_df['avg_price_per_kg'],kde=True)

Our target variable is symmetrical distributed 

In [ ]:
train_df.kurtosis()

We can see our independent variables have large number of outliers

--------------------------------------------------------------------

# 2.1 Data Visualization for outliers detection and removing

Plots to show outliers

In [ ]:
df = ['Weight_Kg', 'Low_Price', 'High_Price', 'Sales_Total', 'Total_Qty_Sold', 'Total_Kg_Sold', 'Stock_On_Hand']
train_df[df].hist(figsize=(10,10));

outliers

In [ ]:
#calculate z
threshold = 3
lowPrice_z = np.abs(stats.zscore(train_df.Low_Price))
lowPrice_abs_z_scores = np.abs(lowPrice_z)
df1 = train_df[lowPrice_abs_z_scores < threshold]

highPrice_z = np.abs(stats.zscore(df1.High_Price))
highPrice_abs_z_scores = np.abs(highPrice_z)                                
df2 = df1[highPrice_abs_z_scores < threshold]

weight_z = np.abs(stats.zscore(df2.Weight_Kg))
weight_abs_z_scores = np.abs(weight_z) 
df3 = df2[weight_abs_z_scores < threshold]

salesTotal_z = np.abs(stats.zscore(df3.Sales_Total))
salesTotal_abs_z_scores = np.abs(salesTotal_z) 
df4 = df3[salesTotal_abs_z_scores < threshold]

totalQtySold_z = np.abs(stats.zscore(df4.Total_Qty_Sold))
totalQtySold_abs_z_scores = np.abs(totalQtySold_z) 
df5 = df4[totalQtySold_abs_z_scores < threshold]

totalKgSold_z = np.abs(stats.zscore(df5.Total_Kg_Sold))
totalKgSold_abs_z_scores = np.abs(totalKgSold_z)
df6 = df5[totalKgSold_abs_z_scores < threshold]

stockOnHand_z = np.abs(stats.zscore(df6.Stock_On_Hand))
stockOnHand_abs_z_scores = np.abs(stockOnHand_z)
df7 = df6[stockOnHand_abs_z_scores < threshold]

avgPricePerKg_z = np.abs(stats.zscore(df7.avg_price_per_kg))
avgPricePerKg_abs_z_scores = np.abs(avgPricePerKg_z)
trainData_df = df7[avgPricePerKg_abs_z_scores < threshold]

Removing outliers

In [ ]:
#Outliears

In [ ]:
df = [ 'Low_Price', 'High_Price', 'Sales_Total', 'Total_Qty_Sold', 'Total_Kg_Sold', 'Stock_On_Hand']
trainData_df[df].hist(figsize=(10,10));

In [ ]:
trainData_df.describe()

In [ ]:
trainData_df.shape, test_df.shape

# 2.2 Dummy Variables

In [6]:
#For The TRAIN DATA (Train_df)
train_df_d = pd.get_dummies(train_df)

train_df_d.columns = [col.replace(" ","_") for col in train_df_d.columns]
train_df_d.drop(['Commodities_APPLE_GOLDEN_DELICIOUS'], axis= 1, inplace =True)

train_df_d.head()

,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg,Province_CAPE,Province_EASTERN_CAPE,...,Date_2020-09-07,Date_2020-09-09,Date_2020-09-16,Date_2020-09-17,Date_2020-09-19,Date_2020-09-21,Date_2020-09-23,Date_2020-09-28,Date_2020-10-01,Date_2020-10-03
1,18.3,150.0,170.0,51710.0,332,6075.6,822,8.51,1,0,...,0,1,0,0,0,0,0,0,0,0
7,11.0,50.0,50.0,16000.0,320,3520.0,0,4.55,1,0,...,0,0,0,0,0,0,0,0,0,0
24,9.0,55.0,55.0,990.0,18,162.0,1506,6.11,0,0,...,0,0,0,0,0,0,0,0,0,0
40,18.3,80.0,120.0,32020.0,388,7100.4,443,4.51,1,0,...,0,0,0,0,0,0,0,0,0,0
69,400.0,1800.0,1800.0,1800.0,1,400.0,2,4.50,0,1,...,0,0,0,0,0,0,0,1,0,0


In [7]:
#For The Test DATA (Test_df)
#using the test data
test_df = test_df[(test_df['Commodities'] == 'APPLE GOLDEN DELICIOUS')]

X_t = pd.get_dummies(test_df)
X_t.columns = [col.replace(" ","_") for col in X_t.columns]
X_t.drop(['Commodities_APPLE_GOLDEN_DELICIOUS'], axis= 1, inplace =True)

X_t.head()



,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,Province_CAPE,Province_EASTERN_CAPE,Province_NATAL,...,Date_2020-09-03,Date_2020-09-07,Date_2020-09-09,Date_2020-09-17,Date_2020-09-19,Date_2020-09-21,Date_2020-09-23,Date_2020-09-28,Date_2020-10-01,Date_2020-10-03
0,12.0,128.0,136.0,5008.0,38,456.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18.3,220.0,220.0,1760.0,8,146.4,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12.0,120.0,120.0,720.0,6,72.0,45,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18.3,160.0,160.0,160.0,1,18.3,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18.3,140.0,160.0,14140.0,100,1830.0,19,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train_df_d.shape, X_t.shape

((1952, 183), (685, 179))

In [ ]:
train_df_d.dtypes

In [ ]:
X_t.dtypes

# 2.3 Correlation and Variable selection

1. Check the independent variables that are highly correleted to the target variable
2. Check those that have moderate correlection
3. Check the weak correlected independent variables
4. Chose variables to remove depending on their relationship with the target variable

we will arrange the data so our target variable be the last column on the dataframe

In [9]:
#Avarage must be the last column (dependent)
#reorder columns
column_titles = [col for col in train_df_d.columns if col!= 'avg_price_per_kg'] + ['avg_price_per_kg']
train_df_d=train_df_d.reindex(columns=column_titles)

The correlection

In [10]:
# Target correlations with features
corrs = pd.DataFrame(train_df_d.corr()['avg_price_per_kg']).rename(columns = {'avg_price_per_kg':'Correlation'})
corrs.sort_values(by='Correlation',ascending=False, inplace=True)
corrs

,Correlation
avg_price_per_kg,1.000000
Container_M4183,0.403229
Size_Grade_1L,0.280966
Province_W.CAPE-BERGRIVER_ETC,0.262051
Size_Grade_1X,0.251451
...,...
Container_JE090,-0.322235
Weight_Kg,-0.337886
Size_Grade_2S,-0.352996
Container_IA400,-0.355565


# Standardazition

1. For Train Data (Train_df thats in df_dummies)

In [11]:
#spliting variables into x and y
X = train_df_d.drop('avg_price_per_kg', axis = 1)
#X = df_dummies.drop('Low_Price', axis = 1)
Y = train_df_d['avg_price_per_kg']



In [12]:
X_t.shape, X.shape

((685, 179), (1952, 182))

2. For Test Data (Test_df thats in X_t dummies)

# MODELS

1. Training in Train Data (Standardized Train Data)

In [ ]:
#X_real = X_real[['Total_Qty_Sold','Stock_On_Hand']]

In [ ]:
#predict_vector = X_standardise[['Total_Qty_Sold','Stock_On_Hand']]

In [ ]:
#Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
#function to calculate the RMSE
def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(y_pred,y_test))

In [ ]:
#Function for test and train
def model_test(model, X, y):
    #Predict
    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y_pred, y))
    print('R2 Score is : {} | Root Mean Square Error is : {}'
          .format(r2,rmse))
    
    return

a. Using The Linear Regression (Train and Test from Train/Test Split)

In [ ]:
#Function for Linear Regression
def linearRegression(X, y):
    global Linear_model
    Linear_model = LinearRegression()
    Linear_model.fit(X, y)
    return Linear_model
#Training the model
linearRegression(X_train, y_train)

In [ ]:
model_test(Linear_model, X_train, y_train)

In [ ]:
model_test(Linear_model, X_test, y_test)

B. Using The Lasso Regression

In [ ]:
#Using Lasso Regression
def lasso_Regression(X, y):
    global Lasso_model
    Lasso_model = Lasso(alpha=0.01)
    Lasso_model.fit(X, y)
    return Lasso_model

lasso_Regression(X_train, y_train)

In [ ]:
model_test(Lasso_model, X_train, y_train)

In [ ]:
model_test(Lasso_model, X_test, y_test)

C. Using Decision Tree

In [ ]:
def decision_Tree(X,y):
    global dec_tree
    dec_tree = DecisionTreeRegressor(random_state=10)
    dec_tree.fit(X, y)
    return dec_tree

decision_Tree(X_train, y_train)

In [ ]:
model_test(dec_tree, X_train, y_train)

In [ ]:
model_test(dec_tree, X_test, y_test)

In [ ]:
model_test(dec_tree, X_test, y_test)

D. Using Random Forest 

In [ ]:
def random_Forest(X,y):
    global rand_forest
    rand_forest = RandomForestRegressor(n_estimators=350, max_depth=450, max_features=40, random_state=3)
    rand_forest.fit(X, y)
    return dec_tree

random_Forest(X_train, y_train)

In [ ]:
model_test(rand_forest, X_train, y_train)

In [ ]:
model_test(rand_forest, X_test, y_test)

In [ ]:
model_test(rand_forest, X_train, y_train)

In [ ]:
model_test(rand_forest, X_test, y_test)

.

E. Using Support Vector Regression Model

In [ ]:
# Define the models which we'll include in our ensemble. 
# We pass a list of tuples, which each have a string identifier for the
# model (arbitrary choice), along the actual instantiated sklearn model.  
models = [("Lasso",Lasso_model),("DT",dec_tree),("RF",rand_forest)]

# Specify weights for weighted model averaging
model_weightings = np.array([0.1,0.3,0.6])
v_reg = VotingRegressor(estimators=models,weights=model_weightings)

In [ ]:
v_reg.fit( X_train, y_train)

In [ ]:
v_reg.fit(X_test, y_test)

In [ ]:
# Instantiate decision tree regression model to use as the base model
d_tree = RandomForestRegressor(n_estimators=300, max_depth=100, max_features=25, random_state=3)
# Instantiate AdaBoostRegressor model with a decision tree as the base model
bst_reg = AdaBoostRegressor(base_estimator = d_tree, n_estimators=700, random_state= 10 )

In [ ]:
bst_reg.fit(X_train, y_train)
bst_reg.fit(X_test, y_test)

In [ ]:
test_ada = bst_reg.predict(X_real)

In [ ]:
test_ada

In [ ]:
test_ada


In [ ]:
from catboost import CatBoostRegressor, FeaturesData, Pool

In [ ]:
model = CatBoostRegressor(iterations=100, learning_rate=1, depth=5)
# Fit model
model.fit( X_train, y_train)
model.fit( X_test, y_test)
# Get predictions
preds = model.predict(X_t)

In [ ]:
preds

In [ ]:
y_pred = v_reg.predict(X_t)
#print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred)))

In [ ]:
test_Tree = dec_tree.predict(X_t)
test_lasso = Lasso_model.predict(X_t)
test_linear = Linear_model.predict(X_t)
test_random = rand_forest.predict(X_t)

In [ ]:
test_Tree

In [ ]:
test_lasso

In [ ]:
test_linear

In [ ]:
test_random

In [ ]:
y_pred

.

FIGURES FOR MODELS

In [ ]:
# Create figure and axes
f, ax = plt.subplots(figsize=(15,5), nrows=1, ncols=4, sharey=True)

# Create list of titles and predictions to use in for loop
pred = [dec_tree.predict(X_test), Lasso_model.predict(X_test), Linear_model.predict(X_test), rand_forest.predict(X_test)]
title = [ 'Decision Tree','Lasso Regression', 'Linear Regression','Random Forest']

# Loop through all axes to plot each model's results 
for i in range(3):
    
    rmse = round(np.sqrt(mean_squared_error(pred[i],y_test)))
    ax[i].set_title(title[i]+"  (RMSE: "+str(rmse)+ ")")
    ax[i].set_xlabel('Actual')
    ax[i].set_ylabel('Predicted')
    ax[i].plot(y_test,y_test,'r')
    ax[i].scatter(y_test,pred[i])

In [ ]:
# create submission dataframe
# Create Dataframe of Order_No and Time from Pickup to Arrival
submission = pd.DataFrame(
    {'Index': test_ID,
     'avg_price_per_kg': test_random
    })

submission.to_csv("RF_Submissions.csv", index=False)